In [1]:
%load_ext autoreload
%autoreload 2
import subprocess
from pathlib import Path

from TPTBox import BIDS_FILE, NII, POI, BIDS_Global_info, Location, calc_poi_from_subreg_vert, to_nii
from TPTBox.core.bids_constants import sequence_splitting_keys
from TPTBox.core.dicom import dicom_extract
from TPTBox.stitching import stitching

# TODO TotalSeg
# TODO Muscle if psoas visible
# TODO Spineps if spine visible
# TODO TotalSpineSeg

#Export from dicom
dicom_dataset:Path|None = None #If already exported from dicom: None
nii_dataset = "/DATA/NAS/ongoing_projects/robert/datasets/dataset-University-Hospital-Brno"
# Stich images make 4D into 3D images
stiching = False # TODO Must be customized, what to stiched. Same is for splitting 4D data into 3D.

# Segmentation
run_spineps = False # Must be installed https://github.com/Hendrik-code/spineps
run_totalvibeseg = True # Must be installed https://github.com/Hendrik-code/spineps

# Analysis
run_cobb_and_lordosis_and_kyphosis = True and run_spineps

In [6]:
nii_dataset = Path(nii_dataset)
### extract dicom ###
if dicom_dataset is not None:
    dicom_extract.extract_folder(dicom_dataset, nii_dataset)

In [7]:
### stitching and splitting in to 3D array ###
# TODO  Must be customized, what to stiched. Same is for splitting 4D data into 3D.
if stiching:
    def stitching2(l: list[NII], out: BIDS_FILE):
        if out.exists():
            return
        if l[0].get_num_dims() == 4:
            for i, name in enumerate(["water", "inphase", "fat"]):
                # TODO assume a 4D image is water/inphase/fat (x,y,z,3). This will not be true in general.
                stitching2(
                    [n.split_4D_image_to_3D()[i] for n in l],
                    out.get_changed_bids(info={"part": name}, parent="rawdata", non_strict_mode=True),
                )
        else:
            assert l[0].get_num_dims() == 3, l[0].shape
            stitching(*l, out=out, dtype=l[0].dtype)
    sequence_splitting_keys.remove("sequ")
    print("Files are splitted by this keywords:",sequence_splitting_keys)
    bgi = BIDS_Global_info(nii_dataset, sequence_splitting_keys=sequence_splitting_keys)
    for _name, subject in bgi.enumerate_subjects():
        q = subject.new_query(flatten=True)
        q.filter_filetype("nii.gz")
        # TODO add filter for things that should never be stichted
        q.filter("sequ", lambda x: "stitched" not in str(x), required=False)
        q.unflatten()

        for i in q.loop_dict(key_addendum=["acq"]): #only "acq" that are equal are stiched
            for k, v in i.items():
                # you must decide witch files to stich. Currently all with the same ending bids_format and ax/iso/sag/cor (e. g. sub-*_acq-ax*_T2w.nii.gz) will be stiched.
                print(k)
                sequ = "stitched"
                if "sequ" in v[0].info:
                    sequ = sequ + "-" + str(v[0].get("sequ",""))
                stitching2(
                    [to_nii(x) for x in v],
                    v[0].get_changed_bids(parent="rawdata", info={"sequ": sequ}, non_strict_mode=True),
                )
                print(i)


In [8]:
if run_spineps:
    from TPTBox.segmentation.spineps import run_spineps_all
    run_spineps_all(nii_dataset)

In [ ]:
if run_totalvibeseg:
    bgi = BIDS_Global_info(nii_dataset, sequence_splitting_keys=sequence_splitting_keys)
    for _name, subject in bgi.enumerate_subjects():
        q = subject.new_query(flatten=True)
        q.filter_filetype("nii.gz")
        q.filter_format(lambda x: x not in ["msk","seg"])
        q.filter("seg",lambda _x: False,required=False)
        for i in q.loop_list():
            out_seg = i.get_changed_path("nii.gz","msk",parent="derivatives",info={"seg":"TotalVibeSeg80"},non_strict_mode=True)
            if out_seg.exists():
                continue
            from TPTBox.segmentation.TotalVibeSeg.inference_nnunet import run_inference_on_file
            run_inference_on_file(80,[to_nii(i)],out_file=out_seg)

In [10]:
from TPTBox.core.bids_constants import formats

result_cobb = {}
if run_cobb_and_lordosis_and_kyphosis:
    ### TODO ###
    ############
    from TPTBox.spine.statistics.angles import plot_cobb_and_lordosis_and_kyphosis
    bgi = BIDS_Global_info(nii_dataset)
    for _name, subject in bgi.enumerate_subjects():
        q = subject.new_query()
        q.filter("seg", lambda x: x in ["spine","subreg"])
        q.filter("seg","vert")
        for fam in q.loop_dict():
            subreg = fam["msk_seg-spine"][0] if "msk_seg-spine" in fam else fam["msk_seg-subreg"][0]
            #dataset / "derivatives_seg/sub-0003106805/T2w/sub-0003106805_sequ-stitched_acq-sag_mod-T2w_part-inphase_seg-spine_msk.nii.gz"
            vert = fam["msk_seg-vert"][0]
            img = vert
            for i in formats:
                if i in fam:
                    img =fam[i][0]
                    break
            poi = calc_poi_from_subreg_vert(
                vert,
                subreg,
                subreg_id=[
                    Location.Vertebra_Direction_Right,
                    Location.Vertebra_Disc_Inferior,
                    Location.Vertebra_Disc,
                    100
                ],
            )
            # Optimal save poi with poi.save
            jpg_out_name = subreg.get_changed_path('jpg','snp',info={'desc':'cobb_and_lordosis_and_kyphosis'})
            a, b, _ = plot_cobb_and_lordosis_and_kyphosis(
                jpg_out_name,
                poi,
                img,
                vert
            )
            result_cobb[fam.family_id] = (a,b)
            print(a, b)


In [11]:
#totalspineseg /DATA/NAS/ongoing_projects/robert/datasets/dataset-University-Hospital-Brno/rawdata/ /DATA/NAS/ongoing_projects/robert/datasets/dataset-University-Hospital-Brno/derivative_totalspineseg --BIDS

In [1]:
from TPTBox.segmentation.oar_segmentator.run import run_oar_segmentor



In [3]:
run_oar_segmentor("/DATA/NAS/datasets_processed/CT_spine/dataset-shockroom-without-fx/rawdata_fixed/sub-CTFU03465/ses-20190725/sub-CTFU03465_ses-20190725_sequ-6_ct.nii.gz",keep_models_loaded=True,override=True)

resample
start


Saving segmentations:   0%|          | 0/9 [00:00<?, ?it/s]

[*] Save /DATA/NAS/datasets_processed/CT_spine/dataset-shockroom-without-fx/rawdata_fixed/sub-CTFU03465/ses-20190725/derivatives/sub-CTFU03465_ses-20190725_sequ-6_seg-oar-259_msk.nii.gz as uint8


Saving segmentations: 100%|██████████| 9/9 [00:09<00:00,  1.05s/it]


[ ] resample_from_to: shp=(258, 251, 367); ori=('R', 'A', 'S'), zoom=(1.5, 1.5, 1.5), seg=True to ((512, 499, 917), array([[-7.55859375e-01,  0.00000000e+00, -0.00000000e+00,
         2.10122070e+02],
       [-0.00000000e+00,  7.55859375e-01, -0.00000000e+00,
         4.23779297e+01],
       [ 0.00000000e+00,  0.00000000e+00,  6.00097656e-01,
        -1.34380005e+03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]]), (0.7558594, 0.7558594, 0.6))
[*] Save /DATA/NAS/datasets_processed/CT_spine/dataset-shockroom-without-fx/rawdata_fixed/sub-CTFU03465/ses-20190725/derivatives/sub-CTFU03465_ses-20190725_sequ-6_seg-oar-combined_msk.nii.gz as uint8
